# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [60]:
# import libraries

import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('tic-tac-toe.csv')

In [3]:
data.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [15]:
# Perform one-hot encoding

categorical_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
encoded_data = pd.get_dummies(data, columns=categorical_columns, prefix=categorical_columns, drop_first=True)
encoded_data = encoded_data.apply(pd.to_numeric)

# relabel the target variable into a numeric one

encoded_data.rename(columns={'class': 'X_wins'}, inplace=True)
encoded_data['X_wins'] = data['X_wins'].replace({True: 1, False: 0})
encoded_data['X_wins'] = pd.to_numeric(data['X_wins'])

encoded_data.dtypes

X_wins    int64
TL_o      uint8
TL_x      uint8
TM_o      uint8
TM_x      uint8
TR_o      uint8
TR_x      uint8
ML_o      uint8
ML_x      uint8
MM_o      uint8
MM_x      uint8
MR_o      uint8
MR_x      uint8
BL_o      uint8
BL_x      uint8
BM_o      uint8
BM_x      uint8
BR_o      uint8
BR_x      uint8
dtype: object

In [18]:
encoded_data.head()

,X_wins,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,0,1,0
1,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,1,0
2,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1
3,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0
4,1,0,1,0,1,0,1,0,1,1,0,1,0,0,0,1,0,0,0


In [20]:
# Separate inputs and outputs

X = encoded_data.drop('X_wins', axis=1)
y = encoded_data['X_wins']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [22]:
# Standardize the input data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [54]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
23/23 [==============================] - 2s 2ms/step - loss: 2.2765 - accuracy: 0.0334 
Epoch 2/5
23/23 [==============================] - 0s 2ms/step - loss: 1.5888 - accuracy: 0.6086
Epoch 3/5
23/23 [==============================] - 0s 2ms/step - loss: 0.7876 - accuracy: 0.6852
Epoch 4/5
23/23 [==============================] - 0s 4ms/step - loss: 0.5915 - accuracy: 0.7242
Epoch 5/5
23/23 [==============================] - 0s 2ms/step - loss: 0.5413 - accuracy: 0.7563


In [55]:
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

8/8 [==============================] - 0s 1ms/step - loss: 0.6393 - accuracy: 0.7125
0.6393190026283264 0.7124999761581421


In [56]:
model.save('tic-tac-toe.model')

INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [68]:
loaded_model = tf.keras.models.load_model('tic-tac-toe.model')
predictions = loaded_model.predict([X_test])

8/8 [==============================] - 0s 1ms/step


In [78]:
print(np.argmax(predictions[10]))

1


In [79]:
y_test.iloc[10]

1

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [97]:
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer=adam_optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
23/23 [==============================] - 1s 1ms/step - loss: 0.9638 - accuracy: 0.5961
Epoch 2/5
23/23 [==============================] - 0s 3ms/step - loss: 0.5333 - accuracy: 0.7075
Epoch 3/5
23/23 [==============================] - 0s 2ms/step - loss: 0.3681 - accuracy: 0.8106
Epoch 4/5
23/23 [==============================] - 0s 2ms/step - loss: 0.1603 - accuracy: 0.9415
Epoch 5/5
23/23 [==============================] - 0s 3ms/step - loss: 0.0712 - accuracy: 0.9708


**Which approach(es) did you find helpful to improve your model performance?**

* Increasing the layers had only a marginal effect
* Increasing the epochs and tweaking the learning time showed a major effect on the model performance